Let's run the python profiler to see what the issue is

In [1]:
import lsst.sims.skybrightness as sb
import profile
import numpy as np
import healpy as hp

In [2]:
sm = sb.SkyModel(mags=True)
nside = 16
hpmap = np.zeros(hp.nside2npix(nside))
lat, ra = hp.pix2ang(nside, np.arange(hpmap.size))
dec = np.pi/2-lat
good = np.where(dec < 0)
dec = dec[good]
ra = ra[good]
print 'computing at %i points on the sky' % ra.size

computing at 1504 points on the sky


In [3]:
mjd = 56948.020683
profile_results = profile.run('sm.setRaDecMjd(ra,dec,mjd)')

         24576 function calls (24575 primitive calls) in 0.103 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8    0.000    0.000    0.000    0.000 :0(abs)
       13    0.000    0.000    0.000    0.000 :0(all)
        6    0.000    0.000    0.000    0.000 :0(argsort)
       39    0.000    0.000    0.000    0.000 :0(array)
        2    0.000    0.000    0.000    0.000 :0(compute)
        4    0.000    0.000    0.000    0.000 :0(concatenate)
        5    0.000    0.000    0.000    0.000 :0(copyto)
        1    0.000    0.000    0.000    0.000 :0(cumsum)
        1    0.000    0.000    0.000    0.000 :0(decode)
     1505    0.002    0.000    0.002    0.000 :0(degrees)
       15    0.000    0.000    0.000    0.000 :0(empty)
        1    0.000    0.000    0.000    0.000 :0(endswith)
     1505    0.003    0.000    0.003    0.000 :0(eq_ecl)
        1    0.000    0.000    0.000    0.000 :0(eqeqx)
        2    0.000    0.000

/Users/yoachim/lsst/DarwinX86/healpy/1.8.1+11/lib/python/healpy-1.8.1-py2.7-macosx-10.5-x86_64.egg/healpy/pixelfunc.py:1045: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if phi == None:


In [4]:
profile.run('sm.returnMags()')

         32 function calls in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 :0(decode)
        1    0.000    0.000    0.000    0.000 :0(endswith)
        1    0.000    0.000    0.000    0.000 :0(getpid)
        4    0.000    0.000    0.000    0.000 :0(isinstance)
        1    0.000    0.000    0.000    0.000 :0(issubclass)
        1    0.000    0.000    0.000    0.000 :0(len)
        1    0.000    0.000    0.000    0.000 :0(open)
        1    0.000    0.000    0.000    0.000 :0(readlines)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        1    0.000    0.000    0.000    0.000 :0(startswith)
        1    0.000    0.000    0.000    0.000 :0(stat)
        1    0.000    0.000    0.000    0.000 :0(strip)
        1    0.000    0.000    0.000    0.000 :0(time)
        1    0.000    0.000    0.000    0.000 :0(utf_8_decode)
        1    0.000    0.000    0.000  

/Users/yoachim/gitRepos/sims_skybrightness/python/lsst/sims/skybrightness/skyModel.py:375: RuntimeWarning: divide by zero encountered in log10
  mags = -2.5*np.log10(self.spec)+np.log10(3631.)


In [6]:
# Looks like the alt az transform is killing me!
from lsst.sims.utils import haversine, _raDecFromAltAz, _altAzPaFromRaDec, Site, ObservationMetaData
telescope =  Site('LSST')
omd = ObservationMetaData(mjd=mjd,site=telescope)
altaz_profile = profile.run('_altAzPaFromRaDec(ra,dec,omd)')

         5823 function calls (5687 primitive calls) in 0.945 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        9    0.000    0.000    0.000    0.000 :0(__new__)
       20    0.000    0.000    0.001    0.000 :0(__reduce_ex__)
        1    0.000    0.000    0.000    0.000 :0(_dtdb)
        1    0.000    0.000    0.000    0.000 :0(_gc2gd)
        1    0.000    0.000    0.000    0.000 :0(_gd2gc)
        1    0.000    0.000    0.000    0.000 :0(_taitt)
        4    0.000    0.000    0.000    0.000 :0(_taiutc)
        1    0.000    0.000    0.000    0.000 :0(_tttai)
        1    0.000    0.000    0.000    0.000 :0(_tttdb)
        1    0.000    0.000    0.000    0.000 :0(_utcut1)
       15    0.000    0.000    0.000    0.000 :0(all)
        1    0.001    0.001    0.001    0.001 :0(altazVector)
       32    0.000    0.000    0.000    0.000 :0(any)
        1    0.000    0.000    0.000    0.000 :0(aoppa)
        1    0.914    0